In [ ]:
# 迁移学习

In [ ]:
import torch
import torchvision
from torch import nn, optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import os
from PIL import Image

In [ ]:
data_list = [16, 48, 27, 238, 12]
data_tensor = torch.Tensor(data_list)

In [ ]:
mean = data_tensor.mean()
std = data_tensor.std()

In [ ]:
mean, std

In [ ]:
normalized = (data_tensor - mean) / std

In [ ]:
normalized.mean(), normalized.std()

In [ ]:
(data_tensor - mean).mean()

In [ ]:
data_transforms = {
    'train':transforms.Compose([
        transforms.Scale(230),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'test':transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

In [ ]:
crop = transforms.Scale(12)
img = Image.open('data/img/20-1.jpg')

In [ ]:
img.size

In [ ]:
crop = transforms.Compose([
    transforms.Scale(230),
     transforms.CenterCrop(224),
 ])

In [ ]:
crop_img = crop(img)

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(crop_img)

In [ ]:
crop_img.size

In [ ]:
data_directory = 'data/CH4'
train_set = datasets.ImageFolder(os.path.join(data_directory, 'train'), data_transforms['train'])
test_set = datasets.ImageFolder(os.path.join(data_directory, 'test'), data_transforms['test'])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=5, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=5, shuffle=True, num_workers=4)

In [ ]:
def imshow(inputs):
    inputs = inputs / 2 + 0.5
    inputs = inputs.numpy().transpose((1,2,0))
    plt.imshow(inputs)
    plt.show()

In [ ]:
inputs, classes = next(iter(train_loader))

In [ ]:
input_new = torchvision.utils.make_grid(inputs)

In [ ]:
imshow(input_new)

In [ ]:
imshow(input_new)

In [ ]:
inputs_n = input_new / 2 + 0.5
inputs_n= inputs_n.numpy().transpose((1,2,0))

In [ ]:
inputs_n.shape

In [ ]:
input_new.shape

In [ ]:
alexnet = models.alexnet(pretrained=True)

In [ ]:
alexnet

In [ ]:
def load_param(model, path):
    if os.path.exists(path):
        model.load_state_dict(torch.load(path))

def save_param(model, path):
    torch.save(model.state_dict(), path)

In [ ]:
for param in alexnet.features:
    param.requires_grad = False

In [ ]:
alexnet.classifier = nn.Sequential(
    nn.Dropout(),
    nn.Linear(256*6*6, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 2),
)

In [ ]:
path = 'pkl_model/alexnet_image.pkl'
path2 = 'pkl_model/alexnet'

In [ ]:
torch.save(alexnet, path2)

In [ ]:
alexnet_2 = torch.load(path2)
alexnet_2.eval()

In [ ]:
save_param(alexnet, path)

In [ ]:
load_param(alexnet, path)

In [ ]:
def train(train_loader, model, epochs, lr=0.0001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.classifier.parameters(), lr=lr)
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 10 == 9:
                print('[Epoch: %d, Batch: %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 10))
                running_loss = 0.0
    print('Finish training')

In [ ]:
train(train_loader, alexnet, 2)

In [ ]:
def test(test_loader, model):
    correct = 0
    total = 0
    for i, data in enumerate(test_loader):
        images, labels = data
        output = model(images)
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy on the test set: %d %%' % (100 * correct/ total))

In [ ]:
test(test_loader, alexnet_2)